# Lent Practical 3: The Stommel Model

In this practical we're going to look at the Stommel Model ([Stommel, 1961](https://doi.org/10.1111/j.2153-3490.1961.tb00079.x)) of ocean thermohaline circulation. In context of the system it represents this is still a very simple model, but it is already complex enough that there is no straightforward analytical solution. We will therefore be using entirely numerical methods to explore this model.

## Model Set-Up

<div>
<img src="https://raw.githubusercontent.com/Quantitative-Environmental-Science/data/main/figures/Stommel.png" width="50%"/>
<p>Figure 1. The Stommel Model of Ocean Circulation
</div>

The model consists of two boxes representing the high-latitude ($H$) and low-latitude ($L$) ocean, which are homogeneous in temperature $T$ and salinity $S$. The model simulates the exchange of water between these two boxes $Q$ driven by density variations between the boxes, which are in turn driven by the temperature and salinity of each box. Changes in salinity are forced by a 'salinity flux', $H$, into each box. Conceptually, this 'salinity flux' represents the balance of evaporation and precipitation at the surface of each box - the high-latitude box is assumed to be a region of net precipitation (i.e. adding fresh water, lowering salinity, $H$ is negative), and the low-latitude box a region of net evaporation (i.e. removing fresh water, increasing salinity, $H$ is positive). This salinity flux establishes a density difference between the boxes, which drives a flow of water between the boxes $Q$, which by definition is considered as poleward surface flow and equatorward deep flow.

### Box Volume

We will set up our model to represent the real-world ocean, with the volume of each box determined by the surface area of the ocean that exists in 'high' and 'low' latitude regions (with a cutoff at ±30° latitude), which is roughly 0.95 and 0.15, respectively:

$$ 
SA_{Ocean} = 3.58\times 10^{14}~\mathrm{m^{2}}\\
SA_L=SA_{Ocean}\times 0.85 \\
SA_H=SA_{Ocean}\times 0.15 \\
V_L=S_L \times D \\
V_H=S_H \times D
$$

where $SA_i$ is surface area, $D$ is the average depth of the ocean ($\mathrm{m}$), $V_i$ is volume ($\mathrm{m^{3}}$).

### Seawater Density

The density of ocean water in each box can be approximated by linear dependencies on temperature and salinity:
$$\rho=\rho_0 -\alpha(T-T_0)+\beta(S-S_0)$$
$\alpha$ is the thermal expansion coefficient, $\beta$ is the haline contraction coefficient, and the subscript ’0’ indicates a reference state.

> **Think:** 
> - Based on the equation above how does an increase in temperature impact the density?
> - How about an increase in salinity?

### Ocean Circulation

Using this relationship, the flow strength, $Q$ ($\mathrm{m^{3}yr^{-1}}$), between the boxes can be related to the density difference by a linear law:

$$
Q = k(\rho_L - \rho_H) \\
Q = k(\alpha(T_L - T_H) - \beta(S_L - S_H)) \\
Q = k(\alpha \Delta T - \beta \Delta S)
$$

where $k$ is a proportionality coefficient to convert the change in temperature and salinity to a volume flux in meters per year. Based on this relationship answer the following questions:<br><br>

> **Think:** 
> - What temperature and salinity conditions are required to have poleward surface flow ($Q>0$)?
> - Which of temperature and salinity are the main drivers in the thermohaline circulation today?
> - Can you think of any local (rather than global) processes which might be having different effects on temperature and salinity and contributing to the thermohaline circulation?

### Temperature Relaxation

The temperature of each box is influenced by the temperature of the overlying atmosphere, relaxing back to the atmosphweric temperature with a characteristic timescale $\tau$:

$$
\frac{dT_i}{dt} = \frac{T_{atm} - T_i}{\tau}
$$

> **Think:** 
> - If temperature in each box were not influenced by the atmosphwere, what would happen to the temperature in each box over time?

### Salinity Flux

The salinity flux, $H$, is a slightly trickier concept. In practice, we can derive it from the NET precipitation - evaporation for a box in $\mathrm{m~yr^{-1}}$. For example, image we have a bucket containing 10L of seawater at 35 g/L salinity, and evaporate 1L of water from it. The salinity of the evaporated water is zero g/L, which effectively means that we have added all the salt from 1L of seawater to the remaining 9L water (i.e. 35 g). We can apply the same logic to calculate the salinity flux in each box, assuming a net precipitation-evaporation of 0.25 m/yr in the high-latitude box, assuming average ocean salinity of 35 g/L. This yields:

$$
H = 0.25 ~ SA_H ~ 35 
$$

Note that $H$ is the same in the high and low latitude boxes because evaporation and precipitation must balance globally.

## Making the Model

We can put all this information together to write down four equations that describe the evolution of the temperature and salinity in each box:

$$
\frac{dT_L}{dt} = \frac{-Q \Delta T + \frac{V_L}{\tau} (T^{at}_L - T_L)}{V_L} \\
\frac{dT_H}{dt} = \frac{Q \Delta T + \frac{V_H}{\tau} (T^{at}_H - T_H)}{V_H} \\
\frac{dS_L}{dt} =\frac{-Q \Delta S + H}{V_L} \\
\frac{dS_H}{dt} =\frac{Q \Delta S - H}{V_H}
$$

Where $\Delta T = T_L - T_H$ and $\Delta S = S_L - S_H$.

The cell below contains definitions of the parameters and initial conditions of the model, and the beginning of a loop to calculate the model code. 


In [ ]:
# import some packages
import numpy as np
import matplotlib.pyplot as plt

# Set model parameters
alpha = 2e-4 # thermal expansion coefficient
beta = 7e-4 # haline contraction coefficient
k = 8.3e17 # proportionality coefficient to convert change in T and S to volume flux in units of m³/year
tau = 2 # Timescale for atmosphere/ocean temperature relaxation in years
TatL = 30. + 273.15 # low latitude atmospheric temperature in Kelvin
TatH = 0. + 273.15 # high latitude atmospheric temperature in Kelvin

# Set the areas and volumes of the boxes
SA_ocean = 3.58e14 # Surface area of the ocean
SA_L = SA_ocean * 0.85 # Low latitude area units m2
SA_H = SA_ocean * 0.15 # High latitude area units m2
D = 3000 # depth of each box in meters
VL = SA_L * D # volume of the low latitude box in m3
VH = SA_H * D # volume of the high latitude box in m3

# Calculate the salinity flux, H
Fw = 0.25 # high latitude precipitation - evaporation in units of m/year 
Sref = 35 # reference salinity in units of g/kg (for calculation of E-P)
H = SA_H * Fw * Sref # Evaporation minus precipitation (E-P)

# Timestepping  parameters
duration = 1000 # duration of the simulation in years
dt = 1.0 # simulation step size in years
steps = int(duration / dt)  # number of timesteps in the model
times = np.linspace(0,duration,steps)

# Create empty arrays for the low and high latitude T+S and volume flux
TL = np.zeros(steps) # Initialize an array for the low latitude temperature
SL = np.zeros(steps) # Initialize an array for the low latitude salinity
TH = np.zeros(steps) # Initialize an array for the high latitude temperature
SH = np.zeros(steps) # Initialize an array for the high latitude salinity
Q = np.zeros(steps) # Initialize an array for the volume flux

# Set initial conditions
TL[0] = 298. # low latitude temperature in degrees K
SL[0] = 37. # low latitude salinity in PSS
TH[0] = 273. # high latitude temperature in degrees K
SH[0] = 33. # high latitude salinity in PSS

############################################### 
# Do your work here
############################################### 

for i in range(1,int(steps)):
    
    # Write the model equations here to:
    # 1. calculate T and S differences 
    # 2. calculate Q from T and S differences
    # 3. calculate fluxes
    # 4. update state variables
    

# after the model has run calculate Q at the final time step

# Finally, Create a figure with three panels with a shared x-axis showing:
# 1. The temperature in each box in Celsius
# 2. The salinity in each box
# 3. The latitudinal flow strength in m³/year

###############################################

> **Think**
> 1. Do the timescales for the temperature and salinity changes match the expected differences based on your understanding of the system above?
> 2. Compared to the Lorenz model you worked on last practical, why are the timescales so much longer?
> 3. Set the y limits on the upper subplot to between 0 and 3, you can see the two different controls on the temperature in the different boxes. What are these?
> 4. Now try changing the atmospheric temperature in the low latitude box to 20 degrees centigrade. What do you observe in terms of the relative impact of temperature and salinity on the flow?

## Perturbing the Model

Next, you're going to see what happens if you disturb the Stommel model from its equilibrium state. This is a common technique in modelling natural systems - 'forcing' the model with an external variable to see how it responds.

To do this, we want to remove the 'stabilisation' phase that you see at the start of the model run above by initialising the model at the equilibrium state. You can do this by looking at the last values of the state variable arrays, for example `TL[-1]` will return you the low latitude temperature at the end of the model run. Copy your code from the cell above, find out what the steady state values are for the temperature and salinity in each box, and use these as the initial conditions for the model. Check you've done this correctly by running this model. You should see negligible changes in the temperature and salinity in each box through the model run because you've started at the equilibrium state.

Once you're happy this is working, we'll start our experiment. We're going to simulate the impact of melting a lot of ice in the high latitude box, leading to a reduction in salinity by 0.002 g/L/yr between years 100-200 of the simulation. To do this, you'll need to add a conditional `if` statement at the end of your `for` loop to modify the high latitude salinity.

For example:

```python
for i in range(1,nt):
    # ... rest of the model code ...
    if 100 <= times[i] < 200:  # if the year is between 100 and 200
        SH[i] -= dt * 0.002  # reduce the salinity in the high latitude box by 0.002 g/L/yr
```

Add this to the model, and see what happens to the temperature, salinity and transport flux.



In [ ]:
# Do it here!

> **Think**
> - How does the decrease in the salinity of the ocean in the high latitudes impact the thermohaline circulation?
> - After the perturbation ends, how does the system recover?
> - Does the system return to the original steady state? 
>   - Why is this?
> - Try increasing the magnitude of the decrease in salinity per year, or the duration of the perturbation. How does the system respond? 
>   - At what point do you get clean non-linear responses in the latitudinal flow strength?
>   - Observe how changes in salinity can impact the global temperature in this model.
> - Why does a change in salinity have an impact on the temperature in the high and low latitudes?
>

# Lab Report Question 2

Finally, using the same methodology that you used in last practical, run a sensitivity analysis for the timescale for atmosphere ocean temperature relaxation ($\tau$) and see how it controls the behaviour of the model. Run your model with $\tau$ values of 2, 20 and 200 years, and plot the resulting temperature, salinity and transport fluxes on a single three-panel figure, like the one you made above.

To make the impact of these parameters on model behaviour clearer, start the model far from steady-state, using the initial conditions:

```python
# Set initial conditions
TL[0] = 298. # low latitude temperature in degrees K
SL[0] = 37. # low latitude salinity in PSS
TH[0] = 273. # high latitude temperature in degrees K
SH[0] = 33. # high latitude salinity in PSS
```

> **Questions**
> 1. How and why does changing $\tau$ impact the thermo-haline circulation flux?
> 2. Is $\tau$ a realistic property to change in the model? What physical processes might control this timescale?
> 3. What other ocean properties and processes would you include if you wanted to make a more realistic model of thermohaline circulation?